<div style="font-size: 200%; font-weight: bold; color: maroon;">Structured Streaming: Kafka</div>
<!-- v. 1.0 - Paulo Villegas, 2021 -->

In [ ]:
"""
Simple example for Structured Streaming & Kafka 

Read messages from a Kafka topic and print them out into a Notebook cell
"""

import sys
import datetime

from pyspark import SparkContext
from pyspark.storagelevel import StorageLevel

In [ ]:
# Check that we have a valid Spark master config
cfg = sc.getConf()
cfg.get('spark.master')

In [ ]:
# Create a couple of Jupyter widgets to receive the streaming output
from ipywidgets import widgets

wgt = { 'label' : widgets.Label(layout=widgets.Layout(width='40em'),value='<waiting>'), 
        'text' :  widgets.HTML(layout=widgets.Layout(width='100%', height='30em', border='2px dashed gray')), 
      }
output_widget = widgets.VBox( [wgt['label'], wgt['text']] )

In [ ]:
# Create the call that will receive the Streaming DataFrame

from io import StringIO

def output2widget(df, epoch_id):    
    """
    Get a DataFrame with the elements received in this batch and render them in the widget
    """
    data = df.collect()
    # Update label widget
    ts = datetime.datetime.now()
    wgt['label'].value = f"[{epoch_id} elements={len(data)}]   [ts={ts}]"
    # Write to text widget. Use a buffer to hold all text
    buf = StringIO()
    buf.write( u'<div style="white-space: pre-wrap; background-color: cornsilk; color: indigo;">' )
    for row in data:
        if row.key is not None:
            buf.write( u'<div style="color: red">Key: {}</div>'.format(row.key) )
        buf.write( row.value + u'<hr/>'  )
    buf.write( u'</div>')
    wgt['text'].value = buf.getvalue()
    buf.close()

In [ ]:
# Define streaming configuration
host = 'cluster1bigdata.ii.uam.es:9092'
# topics: test, books, tweet, meteo, news
topic = "books"

wait = 4
rewind = False

In [ ]:
ds1 = spark.readStream \
           .format("kafka") \
           .option("kafka.bootstrap.servers", host) \
           .option("subscribe", topic) \
           .option("startingOffsets", "earliest" if rewind else "latest") \
           .load()

In [ ]:
ds1.printSchema()

In [ ]:
ds2 = ds1.selectExpr("offset", "CAST(key AS STRING)", "CAST(value AS STRING)")

In [ ]:
# Show the widget
output_widget

In [ ]:
query = ds2.writeStream \
           .foreachBatch(output2widget) \
           .trigger(processingTime=f"{wait} seconds") \
           .start()

In [ ]:
try:
    query.awaitTermination()
except KeyboardInterrupt:
    query.stop()
    wgt['label'].value = '<terminated>'